In [124]:
%pprint

Pretty printing has been turned OFF


In [125]:
lines = [l.strip().split(",") for l in open("agaricus-lepiota.data", 'r').readlines()]
len(lines)

8124

In [126]:
cols = [list(x) for x in zip(*lines)]

In [127]:
keys = [list(set(c)) for c in cols]
keys

[['e', 'p'], ['b', 's', 'x', 'f', 'c', 'k'], ['y', 's', 'f', 'g'], ['b', 'w', 'c', 'y', 'r', 'e', 'u', 'n', 'p', 'g'], ['f', 't'], ['l', 'c', 'y', 's', 'm', 'f', 'a', 'n', 'p'], ['f', 'a'], ['w', 'c'], ['n', 'b'], ['b', 'w', 'h', 'r', 'y', 'o', 'e', 'k', 'u', 'n', 'p', 'g'], ['e', 't'], ['e', '?', 'c', 'r', 'b'], ['f', 's', 'y', 'k'], ['f', 's', 'y', 'k'], ['b', 'w', 'c', 'y', 'o', 'e', 'n', 'p', 'g'], ['b', 'w', 'c', 'y', 'o', 'e', 'n', 'p', 'g'], ['p'], ['w', 'n', 'y', 'o'], ['n', 't', 'o'], ['e', 'l', 'f', 'n', 'p'], ['b', 'w', 'h', 'r', 'y', 'o', 'u', 'n', 'k'], ['c', 'y', 's', 'v', 'a', 'n'], ['l', 'w', 'd', 'm', 'u', 'p', 'g']]

In [128]:
def to_one_hot(labels, keys):
    if len(keys) == 2:
        return [[l == keys[0]] for l in labels]
    return [[l == k for k in keys] for l in labels]


In [129]:
def flatten(l):
    return [c for r in l for c in r]

In [130]:
# employ one hot encoding to blow up the feature space of categorial features.
cols = [to_one_hot(c, k) for c, k in zip(cols, keys)]
data = [list(x) for x in zip(*cols)]

In [131]:
data = [flatten(r) for r in data]
data[0]

[False, False, False, True, False, False, False, False, True, False, False, False, False, False, False, False, False, False, True, False, False, False, False, False, False, False, False, False, False, False, True, True, False, True, False, False, False, False, False, False, False, True, False, False, False, False, True, True, False, False, False, False, False, True, False, False, False, True, False, False, False, True, False, False, False, False, False, False, False, False, True, False, False, False, False, False, False, False, True, True, False, False, False, False, False, True, False, False, False, False, True, False, False, False, False, False, False, False, False, True, False, False, True, False, False, False, False, False, False, False, True, False, False]

In [132]:
data = flatten(data)
print(len(data))

918012


In [133]:
import numpy as np
npdata = np.array(data)
npdata = npdata.reshape((-1, 113))
print(npdata.shape)

npdata = np.float32(npdata)
features = npdata[:, 1:]
labels = npdata[:, :1]

print(features.shape)
print(labels.shape)

np.savez("mushroooms.npz", features=features, labels=labels)

(8124, 113)
(8124, 112)
(8124, 1)


In [134]:
# extract data from the .npz file and run the opencv ann_mlp for classification
import cv2

In [135]:
with np.load("./mushroooms.npz") as data:
    features = data["features"]
    labels = data["labels"]
    
print(labels.shape)
print(features.shape)

labels[:100]

(8124, 1)
(8124, 112)


array([[ 0.],
       [ 1.],
       [ 1.],
       [ 0.],
       [ 1.],
       [ 1.],
       [ 1.],
       [ 1.],
       [ 0.],
       [ 1.],
       [ 1.],
       [ 1.],
       [ 1.],
       [ 0.],
       [ 1.],
       [ 1.],
       [ 1.],
       [ 0.],
       [ 0.],
       [ 0.],
       [ 1.],
       [ 0.],
       [ 1.],
       [ 1.],
       [ 1.],
       [ 0.],
       [ 1.],
       [ 1.],
       [ 1.],
       [ 1.],
       [ 1.],
       [ 0.],
       [ 1.],
       [ 1.],
       [ 1.],
       [ 1.],
       [ 1.],
       [ 0.],
       [ 1.],
       [ 1.],
       [ 1.],
       [ 1.],
       [ 1.],
       [ 0.],
       [ 1.],
       [ 1.],
       [ 1.],
       [ 1.],
       [ 1.],
       [ 1.],
       [ 1.],
       [ 1.],
       [ 1.],
       [ 0.],
       [ 0.],
       [ 1.],
       [ 1.],
       [ 1.],
       [ 1.],
       [ 1.],
       [ 1.],
       [ 1.],
       [ 1.],
       [ 1.],
       [ 1.],
       [ 1.],
       [ 1.],
       [ 1.],
       [ 1.],
       [ 1.],
       [ 1.],
      

In [136]:
layer_sizes = np.int32([112, 32, 1])
model = cv2.ml.ANN_MLP_create()
model.setLayerSizes(layer_sizes)
model.setTrainMethod(cv2.ml.ANN_MLP_BACKPROP)
model.setBackpropMomentumScale(0.0)
model.setBackpropWeightScale(0.001)
model.setTermCriteria((cv2.TERM_CRITERIA_COUNT, 50, 0.01))
model.setActivationFunction(cv2.ml.ANN_MLP_SIGMOID_SYM, 2, 1)

print("Training MLP")
model.train(features, cv2.ml.ROW_SAMPLE, labels)

ret, resp = model.predict(features)


Training MLP


In [137]:
resp.shape

(8124, 1)

In [142]:
((resp - labels) * (resp - labels)).mean() * 100



0.0024270100766443647